1. How to import PySpark and check the version?

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.master("local[1]").appName("SparkAssessment.com").getOrCreate()

print(spark.version)

24/09/24 09:27:56 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/24 09:27:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 09:27:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.2


In [47]:
from pyspark.sql.functions import monotonically_increasing_id,row_number,min,max
from pyspark.sql import Window,functions as F

2. How to convert the index of a PySpark DataFrame into a column?


In [19]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()
df_with_index=df.withColumn("index",monotonically_increasing_id())
df_with_index.show()

#or
window_spec=Window.orderBy("value")
df_index=df.withColumn("index",row_number().over(window_spec)-1)
df_index.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/19 15:24:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


3. How to combine many lists to form a PySpark DataFrame?

In [23]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]
data=list(zip(list1,list2))
df=spark.createDataFrame(data,["c1","c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  b|  2|
|  c|  3|
|  d|  4|
+---+---+



4. How to get the items of list A not present in list B?

In [25]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

rdd1=spark.sparkContext.parallelize(list_A)
rdd2=spark.sparkContext.parallelize(list_B)

res=rdd1.subtract(rdd2)
res.collect()

[2, 1, 3]

5.How to get the items not common to both list A and list B?

In [26]:
res1=rdd2.subtract(rdd1)
res_union=res.union(res1)
res_union.collect()

[2, 1, 3, 6, 8, 7]

6. How to get the minimum, 25th percentile, median, 75th, and max of a numeric column?

In [33]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()
min_val=df.agg(min("age")).collect()[0][0]
max_val=df.agg(max("age")).collect()[0][0]

quantiles=df.approxQuantile("age",[0.25,0.50,0.75],0.01)

print("min value:",min_val)
print("25th quartile:",quantiles[0])
print("median:",quantiles[1])
print("75th quartile:",quantiles[2])
print("max val",max_val)

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+

min value: 10
25th quartile: 20.0
median: 30.0
75th quartile: 50.0
max val 86


7. How to get frequency counts of unique items of a column?

In [51]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()
name_count=df.groupBy("job").count()
name_count.show()


+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
|   Doctor|    1|
| Engineer|    4|
+---------+-----+



8. How to keep only top 2 most frequent values as it is and replace everything else as ‘Other’?

In [52]:
top_2_jobs = name_count.orderBy(F.desc("count")).limit(2).select("job").rdd.flatMap(lambda x: x).collect()

# Replace other jobs with 'Other'
result_df = df.withColumn("job", F.when(df["job"].isin(top_2_jobs), df["job"]).otherwise("Other"))

# Show the resulting DataFrame
result_df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



9. How to Drop rows with NA values specific to a particular column?

In [53]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+



In [55]:
df.dropna(subset="id").show()

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   B| NULL|123|
|   B|    3|456|
+----+-----+---+



10. How to rename columns of a PySpark DataFrame using two lists – one containing the old column names and the other containing the new column names?

In [58]:

df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

old_names = ["col1", "col2", "col3"]

new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

for old_name, new_name in zip(old_names, new_names):
    df = df.withColumnRenamed(old_name, new_name)
print("DataFrame with Renamed Columns:")
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+

DataFrame with Renamed Columns:
+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



11. How to bin a numeric list to 10 groups of equal size?
CONFUSING.

In [4]:
from pyspark.sql.functions import rand,initcap
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))
bucketizer=Bucketizer(splits=[0,10,float("Inf")],inputCol="values",outputCol="Buckets")
df_buck=bucketizer.setHandleInvalid("keep").transform(df)
df_buck.show(5)
df.show(5)

+-------------------+-------+
|             values|Buckets|
+-------------------+-------+
|  0.619189370225301|    0.0|
| 0.5096018842446481|    0.0|
| 0.8325259388871524|    0.0|
|0.26322809041172357|    0.0|
| 0.6702867696264135|    0.0|
+-------------------+-------+
only showing top 5 rows

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows



12. How to create contigency table?

In [15]:
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
# df = spark.createDataFrame(data, ["category1", "category2"])
df.crosstab("category1","category2").sort("category1_category2").show()
# df.show()

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  A|  2|  1|
|                  B|  1|  1|
|                  C|  2|  1|
+-------------------+---+---+



13. How to find the numbers that are multiples of 3 from a column?

In [18]:
from pyspark.sql.functions import rand,col

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

mul_3=df.filter(col("random")%3==0)
mul_3.show()
# Show the DataFrame
# df.show()

+---+------+
| id|random|
+---+------+
|  1|     6|
|  2|     9|
|  3|     3|
|  5|     6|
+---+------+



14. How to extract items at given positions from a column?

In [21]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

item=df.select("random").collect()
res=[item[i][0] for i in pos]
print(res)


+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     6|
|  6|    10|
|  7|     1|
|  8|    10|
|  9|     8|
+---+------+

[7, 7, 10, 6]


15. How to stack two DataFrames vertically ?

In [23]:
# Create DataFrame for region A
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

res_df=df_A.union(df_B)

res_df.show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_3|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



16. How to compute the mean squared error on a truth and predicted columns?

In [27]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
from pyspark.sql.functions import mean, col


data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()
squared_error=df.withColumn("mean-squared-error",(col("actual")-col("predicted"))**2)
mse=squared_error.select(mean("mean-squared-error")).collect()[0][0]
print(mse)

+------+---------+
|actual|predicted|
+------+---------+
|     1|        1|
|     2|        4|
|     3|        9|
|     4|       16|
|     5|       25|
+------+---------+

116.8


17. How to convert the first character of each element in a series to uppercase?

In [5]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])
df.withColumn("name",initcap(df["name"])).show()
df.show()


+-----+
| name|
+-----+
| John|
|Alice|
|  Bob|
+-----+

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+



18. How to compute summary statistics for all columns in a dataframe

In [6]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])
Describe=df.describe()
Describe.show()

+-------+------+-----------------+-----------------+
|summary|  name|              age|           salary|
+-------+------+-----------------+-----------------+
|  count|     5|                5|                5|
|   mean|  NULL|             32.4|          66000.0|
| stddev|  NULL|3.209361307176242|9617.692030835673|
|    min| James|               29|            55000|
|    max|Robert|               37|            80000|
+-------+------+-----------------+-----------------+



19. How to calculate the number of characters in each word in a column?

In [15]:
from pyspark.sql.functions import length, explode,split,lag

In [12]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

# df.show()
df.withColumn("length",length("name")).show()

+-----+------+
| name|length|
+-----+------+
| john|     4|
|alice|     5|
|  bob|     3|
+-----+------+



20 How to compute difference of differences between consecutive numbers of a column?

In [19]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [23]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()
window_spec = Window.orderBy("salary")
df_with_diff = df.withColumn("salary_diff", col("salary") - lag("salary").over(window_spec))

df_with_diff_of_diff = df_with_diff.withColumn("diff_of_diff", col("salary_diff") - lag("salary_diff").over(window_spec))

df_with_diff_of_diff.select("name", "salary", "salary_diff", "diff_of_diff").show()

# window_spec=Window.orderBy("salary")
# salary_diff=df.withColumn("salary_diff",col("salary")-lag("salary").over(window_spec))
# salary_diff_diff=salary_diff.withColumn("salary_diff_diff",col("salary_diff")-lag("salary_diff").over(window_spec))
# df.select("name","age","salary","salary_diff","salary_diff_diff")


+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+



24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 10:02:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 1

+-------+------+-----------+------------+
|   name|salary|salary_diff|diff_of_diff|
+-------+------+-----------+------------+
|  James| 55000|       NULL|        NULL|
| Robert| 60000|       5000|        NULL|
|    Jen| 65000|       5000|           0|
|Michael| 70000|       5000|           0|
|  Maria| 80000|      10000|        5000|
+-------+------+-----------+------------+



21. How to get the day of month, week number, day of year and day of week from a date strings?

In [28]:
data = [("2023-05-18","01 Jan 2010",), ("2023-12-31", "01 Jan 2010",)]
df = spark.createDataFrame(data, ["date_str_1", "date_str_2"])

df.show()
df_with_dates = df.select(
    col("date_str_1"),
    col("date_str_2"),
    to_date(col("date_str_1")).alias("date_1"),
    to_date(col("date_str_2"), "dd MMM yyyy").alias("date_2")
)
df_final = df_with_dates.select(
    "date_str_1",
    "date_str_2",
    dayofmonth("date_1").alias("day_of_month_1"),
    weekofyear("date_1").alias("week_number_1"),
    dayofyear("date_1").alias("day_of_year_1"),
    dayofweek("date_1").alias("day_of_week_1"),
    dayofmonth("date_2").alias("day_of_month_2"),
    weekofyear("date_2").alias("week_number_2"),
    dayofyear("date_2").alias("day_of_year_2"),
    dayofweek("date_2").alias("day_of_week_2")
)
df_final.show()

+----------+-----------+
|date_str_1| date_str_2|
+----------+-----------+
|2023-05-18|01 Jan 2010|
|2023-12-31|01 Jan 2010|
+----------+-----------+

+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+
|date_str_1| date_str_2|day_of_month_1|week_number_1|day_of_year_1|day_of_week_1|day_of_month_2|week_number_2|day_of_year_2|day_of_week_2|
+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+
|2023-05-18|01 Jan 2010|            18|           20|          138|            5|             1|           53|            1|            6|
|2023-12-31|01 Jan 2010|            31|           52|          365|            1|             1|           53|            1|            6|
+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+



22. How to convert year-month string to dates corresponding to the 4th day of the month?

In [34]:
# example dataframe
df = spark.createDataFrame([('Jan 2010',), ('Feb 2011',), ('Mar 2012',)], ['MonthYear'])

df.show()
df.withColumn(
    "date",
    to_date(concat(col("MonthYear"),lit(" 4")),"MMM yyyy d")).show()

+---------+
|MonthYear|
+---------+
| Jan 2010|
| Feb 2011|
| Mar 2012|
+---------+

+---------+----------+
|MonthYear|      date|
+---------+----------+
| Jan 2010|2010-01-04|
| Feb 2011|2011-02-04|
| Mar 2012|2012-03-04|
+---------+----------+



23 How to filter words that contain atleast 2 vowels from a series?

In [36]:
# example dataframe
df = spark.createDataFrame([('Apple',), ('Orange',), ('Plan',) , ('Python',) , ('Money',)], ['Word'])

df.show()
df.filter(col("word").rlike("[aeiouAEIOU]")).show()

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+



24. How to filter valid emails from a list?
Difficulty Level: L3

In [39]:

# Create a list
data = ['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']

# Convert the list to DataFrame
df = spark.createDataFrame(data, "string").toDF("email")
df.show(truncate =False)
df.filter(col("email").rlike("@[a-zA-Z0-9.-]")).show()

+--------------------------+
|email                     |
+--------------------------+
|buying books at amazom.com|
|rameses@egypt.com         |
|matt@t.co                 |
|narendra@modi.com         |
+--------------------------+

+-----------------+
|            email|
+-----------------+
|rameses@egypt.com|
|        matt@t.co|
|narendra@modi.com|
+-----------------+



25. How to Pivot PySpark DataFrame?
Convert region categories to Columns and sum the revenue


In [41]:

# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy("region").sum("revenue").show()



+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      1|    US|   5000|
|2021|      1|    EU|   4000|
|2021|      2|    US|   5500|
|2021|      2|    EU|   4500|
|2021|      3|    US|   6000|
|2021|      3|    EU|   5000|
|2021|      4|    US|   7000|
|2021|      4|    EU|   6000|
+----+-------+------+-------+

+------+------------+
|region|sum(revenue)|
+------+------------+
|    US|       23500|
|    EU|       19500|
+------+------------+



26. How to get the mean of a variable grouped by another variable?


In [42]:

# Sample data
data = [("1001", "Laptop", 1000),
("1002", "Mouse", 50),
("1003", "Laptop", 1200),
("1004", "Mouse", 30),
("1005", "Smartphone", 700)]

# Create DataFrame
columns = ["OrderID", "Product", "Price"]
df = spark.createDataFrame(data, columns)

df.show()
df.groupBy("product").mean("price").show()

+-------+----------+-----+
|OrderID|   Product|Price|
+-------+----------+-----+
|   1001|    Laptop| 1000|
|   1002|     Mouse|   50|
|   1003|    Laptop| 1200|
|   1004|     Mouse|   30|
|   1005|Smartphone|  700|
+-------+----------+-----+

+----------+----------+
|   product|avg(price)|
+----------+----------+
|    Laptop|    1100.0|
|     Mouse|      40.0|
|Smartphone|     700.0|
+----------+----------+



27. How to compute the euclidean distance between two columns?

Compute the euclidean distance between series (points) p and q, without using a packaged formula.

NEWLY LEARNED

In [44]:

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [48]:


# Define your series
data = [(1, 10), (2, 9), (3, 8), (4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (9, 2), (10, 1)]

# Convert list to DataFrame
df = spark.createDataFrame(data, ["series1", "series2"])

df.show()

vecAssembler = VectorAssembler(inputCols=["series1", "series2"], outputCol="vectors")
df = vecAssembler.transform(df)

df = df.withColumn("squared_diff", expr("POW(series1 - series2, 2)"))

df.agg(expr("SQRT(SUM(squared_diff))").alias("euclidean_distance")).show()

+-------+-------+
|series1|series2|
+-------+-------+
|      1|     10|
|      2|      9|
|      3|      8|
|      4|      7|
|      5|      6|
|      6|      5|
|      7|      4|
|      8|      3|
|      9|      2|
|     10|      1|
+-------+-------+

+------------------+
|euclidean_distance|
+------------------+
| 18.16590212458495|
+------------------+



28. How to replace missing spaces in a string with the least frequent character?
NEWLY LEARNED

In [58]:
#Sample DataFrame
df = spark.createDataFrame([('dbc deb abed gade',),], ["string"])
df.show()
char_df=df.select(explode(split(col("string"),"")).alias("character"))
c_count=char_df.groupBy("character").count()
leaast_f_count=c_count.orderBy("count").first()
least_char=leaast_f_count["character"]
df.withColumn("modified_string",regexp_replace(col("string")," ",least_char)).show()
print(leaast_f_count)



+-----------------+
|           string|
+-----------------+
|dbc deb abed gade|
+-----------------+

+-----------------+-----------------+
|           string|  modified_string|
+-----------------+-----------------+
|dbc deb abed gade|dbcgdebgabedggade|
+-----------------+-----------------+

Row(character='g', count=1)


29. How to create a TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) after that having random numbers as values?

In [67]:
start_date = '2000-01-01'
num_saturdays = 10

# Generate the list of Saturdays
saturdays = spark.range(num_saturdays).select(
    date_add(expr(f"to_date('{start_date}')"), (7 * col("id")).cast("int")).alias("date")
)
saturdays.withColumn("value",rand()).show()

+----------+-------------------+
|      date|              value|
+----------+-------------------+
|2000-01-01| 0.8533356514456697|
|2000-01-08|0.41917704247487153|
|2000-01-15| 0.9270108468011555|
|2000-01-22| 0.6706461364211385|
|2000-01-29|0.03827833184571505|
|2000-02-05|0.21819029426202263|
|2000-02-12| 0.6323363822254449|
|2000-02-19| 0.6615840549713079|
|2000-02-26|  0.888588677737099|
|2000-03-04| 0.8076123485275778|
+----------+-------------------+



30. How to get the nrows, ncolumns, datatype of a dataframe?

Get the number of rows, columns, datatype and summary statistics of each column of the Churn_Modelling dataset. Also get the numpy array and list equivalent of the dataframe

In [69]:
from pyspark import SparkFiles

In [75]:

url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling.csv"), header=True, inferSchema=True)

#df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling.csv", header=True, inferSchema=True)

df.show(5, truncate=False)

nrows=df.count()
print("nrows:",nrows)
ncolumns=len(df.columns)
print("nColumns:",ncolumns)

data_types={col:(df.schema[col].dataType)for col in df.columns}
print("datatype:",data_types)
summary_stat=df.describe()
numpy_arr=df.toPandas().to_numpy()
print("numpy array shape: ",numpy_arr.shape)
print("summary statistics:",summary_stat)


24/09/24 11:41:04 WARN SparkContext: The path https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling.csv has been added already. Overwriting of added paths is not supported in the current version.


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname |CreditScore|Geography|Gender|Age|Tenure|Balance  |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|1        |15634602  |Hargrave|619        |France   |Female|42 |2     |0.0      |1            |1        |1             |101348.88      |1     |
|2        |15647311  |Hill    |608        |Spain    |Female|41 |1     |83807.86 |1            |0        |1             |112542.58      |0     |
|3        |15619304  |Onio    |502        |France   |Female|42 |8     |159660.8 |3            |1        |0             |113931.57      |1     |
|4        |15701354  |Boni    |699        |France   |Female|39 |1     |0.0      |2            |0        |0             |93826.63       |

31. How to rename a specific columns in a dataframe?

In [80]:
df = spark.createDataFrame([('Alice', 1, 30),('Bob', 2, 35)], ["name", "age", "qty"])

df.show()

# Rename lists for specific columns
old_names = ["qty", "age"]
new_names = ["user_qty", "user_age"]
for old_name,new_name in zip(old_names,new_names):
    
    df=df.withColumnRenamed(old_name,new_name)
df.show()


+-----+---+---+
| name|age|qty|
+-----+---+---+
|Alice|  1| 30|
|  Bob|  2| 35|
+-----+---+---+

+-----+--------+--------+
| name|user_age|user_qty|
+-----+--------+--------+
|Alice|       1|      30|
|  Bob|       2|      35|
+-----+--------+--------+



32. How to check if a dataframe has any missing values and count of missing values in each column?

In [87]:
from pyspark.sql import functions as F

In [90]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()
null_count=df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_count.show()


+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   0|    2|  2|
+----+-----+---+



33 How to replace missing values of multiple numeric columns with the mean?

MAKE USE OF IMPUTER FUNCTION RATHER USING FILL OR FILLNA

In [93]:
from pyspark.ml.feature import Imputer

In [94]:
df = spark.createDataFrame([
("A", 1, None),
("B", None, 123 ),
("B", 3, 456),
("D", 6, None),
], ["Name", "var1", "var2"])

df.show()
cols=["var1","var2"]
imputer=Imputer(inputCols=cols,outputCols=cols,strategy="mean")
model=imputer.fit(df)
imputed_df=model.transform(df)
imputed_df.show()

+----+----+----+
|Name|var1|var2|
+----+----+----+
|   A|   1|NULL|
|   B|NULL| 123|
|   B|   3| 456|
|   D|   6|NULL|
+----+----+----+

+----+----+----+
|Name|var1|var2|
+----+----+----+
|   A|   1| 289|
|   B|   3| 123|
|   B|   3| 456|
|   D|   6| 289|
+----+----+----+



34. How to change the order of columns of a dataframe?

In [96]:
# Sample data
data = [("John", "Doe", 30), ("Jane", "Doe", 25), ("Alice", "Smith", 22)]

# Create DataFrame from the data
df = spark.createDataFrame(data, ["First_Name", "Last_Name", "Age"])

# Show the DataFrame
df.show()
new_order=["Age","First_Name","Last_Name"]
df.select(*new_order).show()

+----------+---------+---+
|First_Name|Last_Name|Age|
+----------+---------+---+
|      John|      Doe| 30|
|      Jane|      Doe| 25|
|     Alice|    Smith| 22|
+----------+---------+---+

+---+----------+---------+
|Age|First_Name|Last_Name|
+---+----------+---------+
| 30|      John|      Doe|
| 25|      Jane|      Doe|
| 22|     Alice|    Smith|
+---+----------+---------+



35. How to format or suppress scientific notations in a PySpark DataFrame?


In [99]:
# Assuming you have a DataFrame df and the column you want to format is 'your_column'
df = spark.createDataFrame([(1, 0.000000123), (2, 0.000023456), (3, 0.000345678)], ["id", "your_column"])

df.show()
df.withColumn("your_column",format_number("your_column",10)).show()

+---+-----------+
| id|your_column|
+---+-----------+
|  1|    1.23E-7|
|  2|  2.3456E-5|
|  3| 3.45678E-4|
+---+-----------+

+---+------------+
| id| your_column|
+---+------------+
|  1|0.0000001230|
|  2|0.0000234560|
|  3|0.0003456780|
+---+------------+



36. How to format all the values in a dataframe as percentages?

In [105]:
# Sample data
data = [(0.1, .08), (0.2, .06), (0.33, .02)]
df = spark.createDataFrame(data, ["numbers_1", "numbers_2"])

df.show()
# df.withColumn("numbers_2",format_number("numbers_2",100)).show()
df_percentage = df.select(
    [F.concat(F.format_number(F.col(col_name) * 100, 2), F.lit('%')).alias(col_name) for col_name in df.columns]
)
df_percentage.show()

+---------+---------+
|numbers_1|numbers_2|
+---------+---------+
|      0.1|     0.08|
|      0.2|     0.06|
|     0.33|     0.02|
+---------+---------+

+---------+---------+
|numbers_1|numbers_2|
+---------+---------+
|   10.00%|    8.00%|
|   20.00%|    6.00%|
|   33.00%|    2.00%|
+---------+---------+



37. How to filter every nth row in a dataframe?

In [107]:
# Sample data
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3), ("Dave", 4), ("Eve", 5),
("Frank", 6), ("Grace", 7), ("Hannah", 8), ("Igor", 9), ("Jack", 10)]

# Create DataFrame
df = spark.createDataFrame(data, ["Name", "Number"])

df.show()
window_spec=Window.orderBy(monotonically_increasing_id())
df=df.withColumn("rn",row_number().over(window_spec))

n=5

df=df.filter((df.rn%n)==0)
df.show()

+-------+------+
|   Name|Number|
+-------+------+
|  Alice|     1|
|    Bob|     2|
|Charlie|     3|
|   Dave|     4|
|    Eve|     5|
|  Frank|     6|
|  Grace|     7|
| Hannah|     8|
|   Igor|     9|
|   Jack|    10|
+-------+------+



24/09/24 12:38:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:38:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:38:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:38:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:38:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----+------+---+
|Name|Number| rn|
+----+------+---+
| Eve|     5|  5|
|Jack|    10| 10|
+----+------+---+



38 How to get the row number of the nth largest value in a column?

In [109]:


from pyspark.sql import Row

# Sample Data
data = [
Row(id=1, column1=5),
Row(id=2, column1=8),
Row(id=3, column1=12),
Row(id=4, column1=1),
Row(id=5, column1=15),
Row(id=6, column1=7),
]

df = spark.createDataFrame(data)
df.show()

from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

window = Window.orderBy(desc("column1"))
df = df.withColumn("row_number", row_number().over(window))

n = 3 # We're interested in the 3rd largest value.
row = df.filter(df.row_number == n).first()

if row:
    print("Row number:", row.row_number)
print("Column value:", row.column1)


+---+-------+
| id|column1|
+---+-------+
|  1|      5|
|  2|      8|
|  3|     12|
|  4|      1|
|  5|     15|
|  6|      7|
+---+-------+

Row number: 3
Column value: 8


24/09/24 12:40:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:40:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/24 12:40:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


39. How to get the last n rows of a dataframe with row sum > 100?

In [113]:
# Sample data
data = [(10, 25, 70),
(40, 5, 20),
(70, 80, 100),
(10, 2, 60),
(40, 50, 20)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

# Display original DataFrame
df.show()
df_sum=df.withColumn("row_sum",F.col("col1")+F.col("col2")+F.col("col3"))
filtered_df=df_sum.filter(F.col("row_sum")>100)
n=5
last_n_rows=filtered_df.tail(n)
for row in last_n_rows:
    print(row)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|  10|  25|  70|
|  40|   5|  20|
|  70|  80| 100|
|  10|   2|  60|
|  40|  50|  20|
+----+----+----+

Row(col1=10, col2=25, col3=70, row_sum=105)
Row(col1=70, col2=80, col3=100, row_sum=250)
Row(col1=40, col2=50, col3=20, row_sum=110)


40. How to create a column containing the minimum by maximum of each row?


In [125]:
from pyspark.sql.types import *

In [128]:
# Sample Data
data = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

df.show()
# max_df=df.withColumn("maximum_no",F.greatest("col1","col2","col3"))
# min_of_max=max_df.select(F.min("maximum_no")).collect()[0][0]
# print("minimum of maximum by each row is:",min_of_max)
# max_df.show()

def min_max_ratio(row):
    return float(min(row))/max(row)

min_max_ratio_udf=udf(min_max_ratio,FloatType())
df = df.withColumn('min_by_max', min_max_ratio_udf(array(df.columns)))

res_df.show()



24/09/24 14:12:12 ERROR Executor: Exception in task 0.0 in stage 195.0 (TID 177)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_13648/2148936713.py", line 14, in min_max_ratio
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 720, in min
    return _invoke_function_over_columns("min", col)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/column.py", line 65, in _to_java_column
    raise PySparkTypeError(
pyspark.errors.exceptions.base.PySpar

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
|   7|   8|   9|
|  10|  11|  12|
+----+----+----+



PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_13648/2148936713.py", line 14, in min_max_ratio
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 720, in min
    return _invoke_function_over_columns("min", col)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/sql/column.py", line 65, in _to_java_column
    raise PySparkTypeError(
pyspark.errors.exceptions.base.PySparkTypeError: [NOT_COLUMN_